# 認識システムの構築

---

認識システムを構築します。フロントエンドで入力された手書きデータを、学習システムで学習したモデル・重みを利用して認識（推論）します。必要な計算能力やメモリ容量は「学習 >> 推論」であるため、CPU のみの計算資源で構築します。
![構成](images/210.create_rs.png)

今回作成・使用する機械学習システムは、学習、認識などを指示するフロントエンドシステムと実行するシステムが異なっています。フロントエンド（本Jupyter Notebook）から学習システム（1.lerning_systemで構築）や認識システム（本Notebookで構築）に学習、認識等のスクリプトを転送し、これらの実行させます。学習、認識等のスクリプトは `script/tensorflow` フォルダに格納されています。

## VCP の初期化

VCP を利用するために必要なアクセストークンを入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

VCP SDKの初期化します。エラーになった場合、上のセルを `unfreeze` してから再実行し、正しいアクセストークンを入力してください。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

## パラメータ設定

認識システムの構築情報です。

In [ ]:
from pathlib import Path

### 認識システム ######################
# 認識システムのユーザー名（学習システムのユーザーと同一人物と想定）
rs_user = 'user00'
rs_user_pubkey = '~/.ssh/id_rsa.pub'
rs_user_prvkey = '~/.ssh/id_rsa'

# 認識システムのユニットグループ名
rs_ugroup_name = 'TfCpu'

# 認識システムのVCノードのspec
rs_vc_provider = 'aws'       # プロバイダ
rs_vcnode_flavor = 'small'   # フレーバー


### 学習システム ######################
# 学習システム（利用：True, 不使用：False）
training_system = True

# 学習システム構築Notebookの作業フォルダ（training_system = True の時必須）
ohpc_WORK_DIR = ''

# 学習システムの UnitGroup 名（training_system = True の時必須）
ts_ugroup_name = 'TFclusterGPU'

## 認識システムの構築

### 内部パラメータの設定と学習システムのパラメータの読み込み

#### システムパラメータの設定

システム構築情報です。変更しないでください。

In [ ]:
### VC 利用者鍵 ######################
import os
rs_vcu_pubkey = os.path.expanduser('~/.ssh/id_rsa.pub')
rs_vcu_prvkey = os.path.expanduser('~/.ssh/id_rsa')

### 学習システム、認識システム共通 ######################
# TensorFlow 作業ディレクトリ（学習システム、認識システムとも同じパス）
tf_work_dir = 'tensorflow'

### フロントエンド（本Jupyter Notebook）######################
# 学習済みモデル・重みファイルの保存先ディレクトリ
fe_save_dir = './data'

#### 学習システムのパラメータ読み込み

In [ ]:
%run scripts/ts/group.py

if training_system:
    ts_gvars = load_group_vars(ts_ugroup_name, "../1.learning_system/" + ohpc_WORK_DIR)

エラーが発生した場合、`training_system = True` であるにも関わらず、`ohpc_WORK_DIR` や `ts_ugroup_name` に誤りがある場合や、実際にその `作業フォルダ` や `UnitGroup` がないことが考えられます。

### 認識システム用VCノードの作成

#### VCノードのspecの指定とssh関連の設定
学習システムで学習したモデルと重みを利用して、認識（推論）するために必要十分な性能・容量のノードspecを指定します。

In [ ]:
# UnitGroup の作成
rs_unit_group = vcp.create_ugroup(rs_ugroup_name)

# VCノード spec
rs_spec = vcp.get_spec(rs_vc_provider, rs_vcnode_flavor)

# base コンテナ
rs_spec.image = "vcp/base:1.8.1-ubuntu20.04-x86_64"

# VC user ssh keyfiles
rs_spec.set_ssh_pubkey(rs_vcu_pubkey)

# VC user ssh オプション
rs_vcu_ssh_opts = f"-i {rs_vcu_prvkey} -l root"

#### Unitの作成とVCノードの起動
Unitを作成します。Unitを作成すると同時に VCノード（ここでは Amazon EC2インスタンス）が起動します。処理が完了するまで1分半～2分程度かかります。

In [ ]:
# Unitの作成（同時に VCノードが作成される）
rs_unit = rs_unit_group.create_unit('tf-node', rs_spec)

#### 疎通確認
まず、sshのknown_hostsの設定を行います。その後、VCノードに対して`uname -a`を実行し、`ubuntu`が起動していることを確認します。`ubuntu`が起動していない場合は、`spec.image` に誤りがあります。`24.認識システムの削除.ipynb`により環境を削除し、`spec.image`を修正、全てのセルを`unfreeze`してから、最初から再実行してください。

In [ ]:
# unit_group.find_ip_addresses() は UnitGroup内の全VCノードのIPアドレスのリストを返します
rs_ipaddress = rs_unit_group.find_ip_addresses(node_state='RUNNING')[0] # 今は１つのVCノードのみ起動しているので [0] で最初の要素を取り出す
print(rs_ipaddress)

# ssh 設定(1/2) エラー対策のため分割
!touch ~/.ssh/known_hosts
!ssh-keygen -R {rs_ipaddress}    # ~/.ssh/known_hosts から古いホストキーを削除する

In [ ]:
# ssh 設定(2/2) エラー対策のため分割
!ssh-keyscan {rs_ipaddress} >> ~/.ssh/known_hosts    # ホストキーの登録

# システムの確認
!ssh {rs_vcu_ssh_opts} {rs_ipaddress} uname -a

### 認識システムのTensorFlow環境の構築

VCノード上に、TensorFlowのインストールページ(https://www.tensorflow.org/install) で紹介されているコンテナイメージを使用して環境を構築します。

#### ユーザー登録

作成した Unit に TensorFlow コンテナを実行するユーザーを登録します。便宜上このNotebookでは、TensorFlow 環境のユーザーのsshの鍵として、この JupyterNotebook 環境の鍵を使用します。

In [ ]:
# ユーザー追加
!ssh {rs_vcu_ssh_opts} {rs_ipaddress} 'adduser --disabled-login --gecos "" {rs_user}'
!ssh {rs_vcu_ssh_opts} {rs_ipaddress} 'usermod -aG "docker" {rs_user}'

# ユーザー ssh 公開鍵設定
!ssh {rs_vcu_ssh_opts} {rs_ipaddress} mkdir -m 700 /home/{rs_user}/.ssh
!scp -i {rs_vcu_prvkey} {rs_user_pubkey} root@{rs_ipaddress}:/home/{rs_user}/.ssh/authorized_keys
!ssh {rs_vcu_ssh_opts} {rs_ipaddress} chmod 600 /home/{rs_user}/.ssh/authorized_keys
!ssh {rs_vcu_ssh_opts} {rs_ipaddress} chown -R {rs_user}:{rs_user} /home/{rs_user}/.ssh/

# ssh 疎通確認
rs_user_ssh_opts = f'-i {rs_user_prvkey}'
rs_target = f'{rs_user}@{rs_ipaddress}'
!ssh {rs_user_ssh_opts} {rs_target} id

#### TensorFlow コンテナイメージの作成と実行

ユーザーを追加した TensorFlow コンテナイメージを作成し実行します。

In [ ]:
# 最新のTensorFlowコンテナイメージのダウンロード
docker_image = f'tensorflow/tensorflow:latest'
!ssh {rs_user_ssh_opts} {rs_target} docker pull {docker_image}
#!ssh {rs_user_ssh_opts} {rs_ipaddress} docker info

# ユーザーを追加したコンテナイメージの作成
from pathlib import Path
from tempfile import TemporaryDirectory

!ssh {rs_user_ssh_opts} {rs_target} mkdir -p tensorflow-img
out = !ssh {rs_user_ssh_opts} {rs_target} id -u
uid = out[0]
with TemporaryDirectory() as workdir:
    dockerfile = Path(workdir) / 'Dockerfile'
    with dockerfile.open(mode='w') as f:
        f.write(f'''
FROM {docker_image}

ARG USER={rs_user}
RUN useradd -m ${{USER}} -u {uid}
WORKDIR /home/${{USER}}
USER {rs_user}
''')
    !cat {dockerfile}
    !scp {rs_user_ssh_opts} {dockerfile} {rs_target}:tensorflow-img

!ssh {rs_user_ssh_opts} {rs_target} docker build -t tensorflow-{rs_user} tensorflow-img

# コンテナの実行
!ssh {rs_user_ssh_opts} {rs_target} chmod 755 .
!ssh {rs_user_ssh_opts} {rs_target} docker run -td -v /home/{rs_user}:/home/{rs_user} --rm --ipc=host --net=host --name tensorflow-{rs_user} tensorflow-{rs_user}
!ssh {rs_user_ssh_opts} {rs_target} docker ps

### 認識の準備

認識システム上に作業ディレクトリを作成し、認識システムのイメージ処理に必要なモジュールをインストールし、フロントエンドシステム（JupyterNotebook）から認識スクリプト等を転送します。

In [ ]:
# 必要なモジュールのインストール
!ssh {rs_user_ssh_opts} {rs_target} docker exec -t -u {rs_user} tensorflow-{rs_user} pip install pillow
!ssh {rs_user_ssh_opts} {rs_target} docker exec -t -u {rs_user} tensorflow-{rs_user} pip install matplotlib

# 認識システムのスクリプトを格納しているディレクトリ（JupyterNotebookのパス）
rs_scripts = './scripts/recognition'

# VCノード上の作業ディレクトリの作成
!ssh {rs_user_ssh_opts} {rs_target} mkdir -p {tf_work_dir}

# VCノードで実行するスクリプトの転送
!scp {rs_user_ssh_opts} -qp {rs_scripts}/*.py {rs_target}:{tf_work_dir}

### 認識システムのパラメータ保存

In [ ]:
%run scripts/ts/group.py

update_group_vars(
    rs_ugroup_name,
    rs_ugroup_name=rs_ugroup_name,
    rs_vcu_pubkey = rs_vcu_pubkey,
    rs_vcu_prvkey = rs_vcu_prvkey,
    rs_vc_provider=rs_vc_provider,
    rs_vcnode_flavor=rs_vcnode_flavor,
    rs_ipaddress = rs_ipaddress,
    rs_user = rs_user,
    rs_user_pubkey=str(Path(rs_user_pubkey).expanduser()),
    rs_user_prvkey=str(Path(rs_user_prvkey).expanduser()),
    tf_work_dir = tf_work_dir,
    fe_save_dir = fe_save_dir,
    training_system = training_system,
)

if training_system:
    update_group_vars(
        rs_ugroup_name,
        ohpc_WORK_DIR = ohpc_WORK_DIR,
        ts_ugroup_name = ts_ugroup_name,
    )